# Project Adv ML

In [1]:
import numpy as np 
import matplotlib.pyplot as plt 
import scipy
from scipy import special
import pandas as pd

### Setup

In [2]:
##### PARAMS ####
# k = number of topics
# N = Number of Words in a Document (different for each doc)
# M = Number of Docs
# z_n = [k]-vector = topic distribution for word n 
# Theta = [k]-dim vector = mixture weights

# Gamma = k-dim vector = Determines Theta in Variational Model
# Phi = phi_1 .. phi_N = [N x k] Matrix = it determines the probability distribution for topics z of words in Variational Model

# alpha = [k] - vector = prior probability for theta (mixture weights) (alpha>0)
# beta = [k x V] - Matrix = with beta_ij = p( w^j = 1 | z^i = 1) = prob of for a specific word j given a specific topic i

# D = list of [VxN]-dim matrices, that is M long = [\mathbf{w}_1, ... \mathbf{w}_M], where \mathbf{w}=[w_1,...,w_N] is VxN (one document consisting of N words) 

## Load the Document Corpus
Download the data from 
https://github.com/Blei-Lab/lda-c/blob/master/example/ap.tgz

We can directly load the file "ap.dat" which contains:

1 line = 1 document,

[number of different words in doc] [word index (where the one is in w_n]:[how often it occurs in the doc] [word index 2]:[occurences 2] ...

In [13]:
V = 10473 # Vocab Size given by vocab_list.txt

In [14]:
data = pd.read_csv('ap/ap.dat', sep="#", names=['A'])
data = data.A.str.split(' |:', n=V, expand=True)
data_np = data.values

In [15]:
M = data_np.shape[0] # number of documents
#V = data_np.shape[1] # vocab list # actually this should be loaded

In [17]:
D = []  # this has dim: M x 2 (words and counts) x N_words(document)
for w in range(M): # for each document
    doc_string = data_np[w,:]
    #print(len(doc_string))
    word_indices = doc_string[1::2]
    #print("word_indices", word_indices)
    counts = doc_string[2::2]
    #if w==1:
    #  print(len(word_indices), len(counts))
    #print("counts", counts)
    N_different_words_in_this_doc = doc_string[0] # not needed.
    D.append([])
    all_words = []
    for n,word in enumerate(word_indices):
        if word==None:
            break
        for repetition in range(int(counts[n])):
            #if int(counts[n])>1:
            #  print(word, counts[n])
            all_words.append(int(word))
        D[-1].append(all_words)
    # NOTE: D does not contain M docs with each N_d words, where each word w_n is a V-dim vector.
  # BUT:              it has M docs with each N_d words, where each word is just the unique index v, that is one of the V-dim vector.
  # Reason: V=10,000...




## Remove the standard words

In [18]:
# I think this is already done in the "ap.dat"


### Variational Inference (the E-Step)

In [30]:
def updateGammaPhi(k, alpha, beta, w, tol=10**(-5), MAX_STEPS=100):
    # This implements the update equations for Gamma and Phi for a variational inference step.
    # Inputs: k=int=nr of topics;  alpha=[k]-vec=priors_of_theta(mixture_weights);  beta=[NxV]-matrix=prob_of_word_v_given_topic_
    # Init
    N = len(w)
    #print(np.shape(alpha), np.shape(beta), k, np.shape(D))
    phi = 1/k * np.ones([N,k])
    gamma = alpha + np.ones([k]) * N/k 
    q_old = compute_lower_bound_likelihood(w,phi,gamma, alpha, beta)
    converged = False
    #Loop
    security_count=0
    while not converged and security_count < MAX_STEPS:
        security_count+=1
        phi_old = np.copy(phi)
        gamma_old = np.copy(gamma)
        for n in range(N):
            for i in range(k):
                w_n = w[n] ## this is not a V-Vector, but just the index of the word in the vocab
                #beta_iv is p(w_n^v = 1 | z^i = 1) 
                v = w_n   #unique v for each word w_n
                phi[n,i] = beta[i,v] * np.exp(psi(gamma_old[i])- psi(np.sum(gamma_old)))
                # NOTE: In .c file, they use: phi[n,i] = Psi(gamma[i]) + log_prob_w[i][v], then (log)-sum??, then take exp(phi[n,k] - logsum(phi))
            #normalize Phi
            phi[n,:] = phi[n,:]/np.sum(phi[n,:])
        for i in range(k):
            gamma[i] = alpha[i] + np.sum(phi[:,i])
        # in .c file: they also use a very different formula.
        
        # Convergence criterion is: lower bound of likleihood
        q_new = compute_lower_bound_likelihood(w,phi,gamma, alpha, beta)
        if abs(q_new-q_old)<tol:
            converged = True
        else: 
            q_old=np.copy(q_new)
    return q_new, gamma, phi

def psi(gamma_i):
    # this is the first derivative (via Taylor approximation) of the log \Gamma function
    # according to Wikipedia this is the "digamma" function
    return scipy.special.digamma(gamma_i)

def lgamma(gamma_i):
    return scipy.special.loggamma(gamma_i)


In [31]:
# USE OF LAMBDA
a=np.ones(19)
f = lambda i: a[i]
f(2)

1.0

In [32]:
def compute_lower_bound_likelihood(w,phi,gamma, alpha, beta, verbose=False):
    # This calculate the lower bound of L(gamma, phi, alpha, beta)
    N = len(w)
    global V
    # This corresponds to equation 15 in the paper in Appendix 3.
    loggamma_sum = lambda x: np.log(scipy.special.gamma(np.sum(x)))
    loggamma_x_i = lambda x, i: np.log(scipy.special.gamma(x[i]))
    E_log_thetai_givenGamma = lambda i:  (psi(gamma[i]) - psi(np.sum(gamma))) 

    L = loggamma_sum(alpha) - loggamma_sum(gamma)
    if verbose: print("#0: ", L, end=", ")
    for i in range(k):
        L += -loggamma_x_i(alpha,i) + (alpha[i]-1)*E_log_thetai_givenGamma(i)
        L += +loggamma_x_i(gamma,i) - (gamma[i]-1)*E_log_thetai_givenGamma(i)
        if verbose: print("#",i,": ", L, end=", ")
        for n in range(N):
            L+= phi[n,i] * E_log_thetai_givenGamma(i)
            L+= - phi[n,i] *np.log(phi[n,i])
            #for j in range(V):
            #  L+= phi[n,i]*... easier below, 
            # since sum over j=1..V w_n^j gives only one contribution (unique v for word w_n
            v = w[n] # here w_n is not a vector
            L+= phi[n,i] * np.log(beta[i,v]) 
            if verbose: print("#",i,",",n,": ", L, ", v=",v, end=", ")
 
    if verbose: print(L)
    return L
    

In [33]:
def E_step(k, alpha,beta, MAX_COUNTER=1000):
    gamma_list = []
    phi_list = []
    counter=0
    likelihood_list = []
    for doc in D:
        counter+=1
        # doc is a N_d Vector containing indices of all words
        #print("DOCUMENT, ", len(doc), len(doc[0]))
        likelihood, gamma, phi = updateGammaPhi(k,alpha,beta,doc[0])
        gamma_list.append(gamma)
        phi_list.append(phi)
        likelihood_list.append(likelihood)
        # Stop this if sufficient statistics (how do i know this?)
        if counter%10 ==0:
            print(gamma, phi, likelihood)
        # calculate the approximate q(theta, z | gamma, phi)
        if counter> MAX_COUNTER:
            break
    return likelihood_list, gamma_list, phi_list


## M-Step

In [34]:
def M_step():
  

SyntaxError: unexpected EOF while parsing (<ipython-input-34-bc2016d8a2a2>, line 2)

## Initialisation

In [35]:
k= 10
V= 10473

alpha=np.ones([k])/k

beta = np.zeros([k,V])+0.001


In [36]:
alpha

array([0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1])

In [ ]:
likelihood_list, gamma_list, phi_list = E_step(k,alpha,beta)

/home/peter/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:32: RuntimeWarning: invalid value encountered in double_scalars


In [ ]:
print(likelihood_list)